In [ ]:
# // installing kaggle library

! pip install kaggle

In [ ]:
# configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#importing twiiter sentimnet dataset
#API TO FETCH THE DATASET FROM KAGGLE

!kaggle datasets download -d kazanova/sentiment140





Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 183MB/s]
100% 80.9M/80.9M [00:00<00:00, 178MB/s]


In [ ]:
#EXTRACT THE COMPRESSED FILE
#it will extract all the  components of the comprsssed file and stroe it into the  parent directory
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [ ]:
#here we goining to import some dependencies to process the sentiment analysis

import numpy as np
import pandas as pd   #pandas used to create dataframe
import re  # r stand fro regular expression it is used order to do pattern matching
from nltk.corpus import stopwords #The nltk.corpus.stopwords module in the Natural Language Toolkit (NLTK) library is used to access a list of common stopwords in various languages
from nltk.stem.porter import PorterStemmer #Stemming is the process of reducing words to their root or base form (also called a stem), which helps in normalizing text by removing inflectional forms.
from sklearn.feature_extraction.text import TfidfVectorizer  # used to convert textual data to numerical data
from sklearn.model_selection import train_test_split #used to split our original data into training data and test data where data is used for training of model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # used to calculate the accuracy and performence of model


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#prnting stopwords in english
print(stopwords.words('english'))

#stopwords generally does not have more contextual importance so , machine learning doest not required stopwords for processing


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')



In [ ]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

there are 16 lakh data means 16lakh rows and 6 columns are there

In [ ]:
#printing first five rows of dataframe
twitter_data.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
from tkinter.font import names
#naming the columns and reading the datatset again

columns_name =['target' ,  'id' ,'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = columns_name , encoding = 'ISO-8859-1')


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()
#here target = 0 means => negative tweet and target = 4 => means positive tweet

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#here we are checking or counting the numbers of  missing values or missing text in dataset
twitter_data.isnull().sum()


,0
target,0
id,0
date,0
flag,0
user,0
text,0


so there are no missing values of text in dataset

In [ ]:
#checking the distribution of target column

twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


here there are 800000 negative tweets and 800000 positive tweets are there our dataset doest not have  neutral tweets

In [ ]:
#here we are converting 4 => 1 for postive tweets and  0 will be remains 0 for negative tweets
twitter_data.replace({'target' : {4:1}}, inplace= True)

In [ ]:

twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


negative tweet => 0 and
postive tweet => 1

In [ ]:
#stemming is the process of reduncing the word to its root word

port_stem = PorterStemmer()


In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]' , ' ' , content) # if the tweet content doest not blong to a-zA-Z then  remove from the dataset
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content